# Análise Comparativa de Modelos

In [21]:
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

## 1. Obtenção do Conjunto de Dados

In [22]:
# Obtemos os arquivos brutos de dados e o dicionário antes de iniciar as etapas do pré-processamento.
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,total_bill,valor total da conta em dólares,quantitativa,continua
1,tip,valor da gorjeta em dólares,quantitativa,continua
2,sex,sexo do cliente,qualitativa,nominal
3,smoker,se o cliente fuma,qualitativa,nominal
4,day,dia da semana,qualitativa,ordinal
5,time,horario,qualitativa,ordinal
6,size,pessoas na mesa,quantitativa,discreta


## 2. Limpeza de Dados:

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

### 2.1. Tratamento de dados faltantes:

In [23]:
# Contagem total de dados  faltantes
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

**Observação**: No conjunto de dados não há dados faltantes.

### 2.2. Tratamento de dados discrepantes:

In [24]:
# Separar as variáveis.
target_column = 'tip'

# Organizar as colunas.
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
ordinal_day_columns = (
    df_dict
    .query("variavel in ['day']")
    .variavel
    .to_list()
)
ordinal_time_columns = (
    df_dict
    .query("variavel in ['time']")
    .variavel
    .to_list()
)
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [25]:
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler())  # Normalização de dados contínuos
])
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # Codificação de variáveis
    ('normalization', StandardScaler()) # Normalização de dados
])
ordinal_day_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # Codificação de variáveis
    ('normalization', StandardScaler()) # Normalização de dados
])
ordinal_time_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # Codificação de variáveis
    ('normalization', StandardScaler()) # Normalização de dados
])
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler()) # Normalização de dados
])
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('ordinal_day', ordinal_day_preprocessor, ordinal_day_columns),
    ('ordinal_time', ordinal_time_preprocessor, ordinal_time_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

model = LogisticRegression()